In [52]:
from configparser import ConfigParser
import psycopg2

In [53]:
def config(filename='database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    db_params = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db_params[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db_params

In [54]:
def connect(function, *func_params):
    
    params = config()
    connection = psycopg2.connect(**params)
    global cur
    cur = connection.cursor()
        
    function(*func_params)

    cur.close()
    connection.close()


In [144]:
class DBTable:
    def __init__(self,  table_name, schema_name="public"): # db_conn?
        if not self.check_if_table_exists(table_name, schema_name):
            raise AttributeError(f"The table {table_name} doesn't exist in schema '{schema_name}' or you don't have an access.")
        self._table_name = table_name
        self._column_names = get_column_names(table_name)
        self._unique_columns = get_unique_columns(table_name)
        
    def get_record(self, column_name, value):
        s = f"SELECT * FROM {self._table_name} where {column_name}='{value}'"
        cur.execute(s)
        result = cur.fetchone()
        return result
    
    def check_if_table_exists(self, table_name, schema_name):
        s = f""" SELECT EXISTS (
       SELECT FROM information_schema.tables 
       WHERE  table_schema = '{schema_name}'
       AND    table_name   = '{table_name}'
       ); """
        cur.execute(s)
        result = cur.fetchone()[0]
        return result
    
    def get_unique_columns(self):
        s = f"""SELECT c.column_name 
            FROM information_schema.key_column_usage AS c
            LEFT JOIN information_schema.table_constraints AS t
            ON t.constraint_name = c.constraint_name
            WHERE t.constraint_type in ('PRIMARY KEY', 'UNIQUE') 
            and t.table_name = '{self._table_name}';

            """
        cur.execute(s)
        result = tuple(i[0] for i in cur.fetchall())
        return result
    
    def get_column_names(self):
        s = f"""SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{self._table_name}';"""
        cur.execute(s)
        column_names = cur.fetchall()

        return tuple(item[0] for item in column_names)
      
    
    @property
    def table_name(self):
        return self._table_name
    
    @property
    def column_names(self):
        return self._column_names
    
    @property
    def unique_columns(self):
        return self._unique_columns
    
    def __call__(self, column_name, value):
        if column_name in self.unique_columns:
            record = self.get_record(column_name, value)
            if not record: # если None из таблицы и если нет такого значения - проверить
                raise AttributeError(f"There is no '{value}' value in '{column_name}' column.")
            record_dict = dict(zip(self._column_names, record))
           
            
            return DBRecord(record_dict)
        if column_name in self.column_names:
            raise AttributeError(f"Column '{record_id}' doesn't have unique or primary constraint and cannot be used as identifier.")
        raise AttributeError(f"Table {self._table_name} doesn't have column {column_name}")    
        
        

In [145]:
class DBRecord:
    def __init__(self, db_record_dict):
        super().__setattr__("_record", db_record_dict)
        
    @property
    def fields(self):
        return tuple(self._record.keys())
        
    def __getattr__(self, name):
        if name in super().__getattribute__("_record"):
            return self._record[name]
        raise AttributeError(f"Field name '{name}' doesn't exist.")
        
    def __setattr__(self, name, value):
        if name in super().__getattribute__("_record"):
            return NotImplemented
            # connect to database
        else:
            raise AttributeError(f"Field name '{name}' doesn't exist.")
            
            
        

In [146]:
conn = None
params = config()

conn = psycopg2.connect(**params)

global cur
cur = conn.cursor()

In [147]:
vk_chat = DBTable("vk_chat")
auth_users = DBTable("auth_user")
vk_chat_01 = vk_chat("chat_id", 2_000_000_001)

In [148]:
vk_chat_01.local_id

1

In [149]:
user_01 = auth_users("id", 1)
user_01.username

'valerie'

In [150]:
user_01.__dict__

{'_record': {'id': 1,
  'password': 'pbkdf2_sha256$150000$5JTSYIra7A9S$3wZbCzrfrBDAtoUVyJfjVnHQp/S6qDEJ511ElBQ9uD0=',
  'last_login': datetime.datetime(2020, 9, 21, 13, 31, 57, 649734, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)),
  'is_superuser': True,
  'username': 'valerie',
  'first_name': '',
  'last_name': '',
  'email': 'lera_shel@mail.ru',
  'is_staff': True,
  'is_active': True,
  'date_joined': datetime.datetime(2019, 11, 21, 13, 39, 57, 442258, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None))}}

In [151]:
user_01.id = 111
user_01.id

1

In [152]:
user_01.fields

('id',
 'password',
 'last_login',
 'is_superuser',
 'username',
 'first_name',
 'last_name',
 'email',
 'is_staff',
 'is_active',
 'date_joined')

In [153]:
    
cur.close()

if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.
